In [1]:
from draftbot_sim_adapted import *
import sqlite3
import pandas as pd
import numpy as np
import json
import os

In [2]:
draft_path = 'C:/Users/trist/mtg_research/test_draft.txt'
cards_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/m20/m20-cards.json'
weights_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/new_generated_weights.json'
#Paths to the db files holding our outputs
ss_db_path = 'SingleStrat.sqlite'
ms_db_path = 'MultiStrat.sqlite'

agents = [
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

In [3]:
draft_ss = Draft(n_drafters=8,
        n_rounds=3,
        n_cards_in_pack=14,
        cards_path=cards_path,
        card_values_path=weights_path)

draft_ms = MultiStratDraft(
        n_rounds=3,
        n_cards_in_pack=14,
        cards_path=cards_path,
        card_values_path=weights_path,
        packs_input_file=draft_path,
        agent_list=agents)

In [5]:
draft_ss.draft()
draft_ss.write_to_database(ss_db_path)
if os.path.exists(draft_path):
    os.remove(draft_path)
save_packs_array(draft_ss,draft_path)
draft_ms.draft()
draft_ms.write_to_database(ms_db_path)

C:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [6]:
ss_conn = sqlite3.connect(ss_db_path)
ms_conn = sqlite3.connect(ms_db_path)

ss_picks_df = pd.read_sql("Select * From Picks",ss_conn)
ss_options_df = pd.read_sql("Select * From Options",ss_conn)
#ss_passes_df = pd.read_sql("Select * From Passes",ss_conn)
ss_prefs_df = pd.read_sql("Select * From Preferences",ss_conn)

In [7]:
ms_picks_df = pd.read_sql("Select * From Picks",ms_conn)
ms_options_df = pd.read_sql("Select * From Options",ms_conn)
ms_pass_df = pd.read_sql("Select * From Passes",ms_conn)
ms_prefs_df = pd.read_sql("Select * From Preferences",ms_conn)

In [8]:
mpdf = ms_picks_df.drop(columns=['pick_number','drafter','draft_id'])
spdf = ss_picks_df.drop(columns=['pick_number','drafter','draft_id'])


msof = ms_options_df.drop(columns=['pick_number','drafter','draft_id'])
ssof = ss_options_df.drop(columns=['pick_number','drafter','draft_id'])

ms_pass_df = ms_pass_df.drop(columns=['pick_number','drafter','draft_id'])

ms_prefs_df = ms_prefs_df.drop(columns=['pick_number','drafter','draft_id'])
ss_prefs_df = ss_prefs_df.drop(columns=['pick_number','drafter','draft_id'])

In [24]:
pick_to_explore = 105

print('pick:',np.where(mpdf.iloc[pick_to_explore]==1)[0])
print('options:',np.where(msof.iloc[pick_to_explore]==1)[0])
print('passes:',np.where(np.sum(ms_pass_df.iloc[0:pick_to_explore],axis=0)!=0)[0])

print('pick',np.where(spdf.iloc[pick_to_explore]==1)[0])
print('options:',np.where(ssof.iloc[pick_to_explore]==1)[0])
#print(ss_prefs_df.iloc[35])

pick: [153]
options: [  5  56 110 117 153 186 286]
passes: [  0   5   9  11  14  20  21  22  23  26  27  28  30  33  34  35  36  40
  43  44  49  50  53  55  56  58  62  64  66  67  69  70  71  72  73  74
  77  78  80  82  90  91  93  94  95  96  97  99 101 102 103 106 107 108
 110 112 114 116 117 118 120 121 127 128 129 131 132 133 134 135 136 139
 142 143 145 146 148 151 153 156 157 158 160 169 171 172 173 174 176 179
 181 183 184 185 186 189 190 193 196 198 200 201 202 204 205 206 207 210
 213 215 216 219 220 221 222 223 224 226 229 233 234 235 236 242 243 244
 245 247 250 260 261 266 269 271 273 274 275 276 277 278 282 284 285 286
 287 289 290 292 296 298 300 301 304 306 311 318 324 327]
pick [117]
options: [  5  56 110 117 153 186 286]


In [9]:
ms_conn.close()
ss_conn.close()